In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.optimizers import Adam

os.environ["CUDA_VISIBLE_DEVICES"]='3'
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

image_data_gen = tf.keras.preprocessing.image.ImageDataGenerator()

train_path = "C:/Users/shubh/OneDrive/Desktop/IITP/English"
val_path = "C:/Users/shubh/OneDrive/Desktop/IITP/English"
test_path = "C:/Users/shubh/OneDrive/Desktop/IITP/cv/aditya boudh/e/db"

training_set = image_data_gen.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16,
    shuffle=True
)

validation_set = image_data_gen.flow_from_directory(
    directory=val_path,
    target_size=(256, 256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16
)

test_set = image_data_gen.flow_from_directory(
    directory=test_path,
    target_size=(256, 256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16
)

input = tf.keras.layers.Input(shape=(256, 256, 3))
x = tf.keras.applications.inception_v3.preprocess_input(input)
base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3),
    pooling=None
)

output = base_model(x)
output = Flatten()(output)
output = Dense(33, activation='softmax')(output)
model = tf.keras.Model(inputs=input, outputs=output)
model.summary()

num_epochs = 100
checkpoint_filepath = "./checkpoint"
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=20, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.000001, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor="val_accuracy", save_best_only=True, save_weights_only=True)
]
history = model.fit(
    training_set,
    epochs=num_epochs,
    validation_data=validation_set,
    callbacks=callbacks
)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('InceptionV3 Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.savefig('./loss.png')
plt.show()
plt.close()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('InceptionV3 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.savefig('./accuracy.png')
plt.show()
plt.close()

model.load_weights(checkpoint_filepath)
_, accuracy = model.evaluate(test_set)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
with open('./results.txt', 'a') as f:
    f.write(f"Test accuracy: {round(accuracy * 100, 2)}%.\n")

Num GPUs Available:  0
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLam  (None, 256, 256, 3)       0         
 bda)                                                            
                                                                 
 tf.math.subtract_1 (TFOpLa  (None, 256, 256, 3)       0         
 mbda)                                                           
                                                                 
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 73728)         

ValueError: Asked to retrieve element 0, but the Sequence has length 0